In [47]:
from keras.models import load_model
from utils import CircularStack
import airsim
import cv2
import numpy as np
import time

In [29]:
model = load_model('model.h5df')

In [18]:
client = airsim.CarClient()
client.confirmConnection()
client.enableApiControl(True)


Connected!
Client Ver:1 (Min Req: 1), Server Ver:1 (Min Req: 1)



In [52]:
cstack = CircularStack(3)

# def stackToInput(stack):
#     return np.concatenate([i for i in stack])
    
cstack.update(2)
cstack.update(3)
cstack.update(4)

print(list(cstack))

TypeError: 'CircularStack' object is not iterable

In [22]:
car_controls = airsim.CarControls()
start_time = time.time()
while (time.time() - start_time) < 2:
    
    response = client.simGetImages([airsim.ImageRequest(0, airsim.ImageType.Scene, False, False)])[0]
    img = np.frombuffer(response.image_data_uint8, dtype=np.uint8)
    img = img.reshape(response.height, response.width, 4)[..., :3]
    img = cv2.resize(img, (128, 128))
    print(img.shape)
    
    
#     prediction = model.predict(img)
#     steering_angle = predictionToSteeringAngle(prediction)

#     car_controls.throttle = 0.4
#     car_controls.steering = steering_angle
    
#     client.setCarControls(car_controls)
    
#     print("Sending steering {s}".format(s=steering_angle))
    time.sleep(0.1)
    
client.enableApiControl(False)

(128, 128, 3)
(128, 128, 3)
(128, 128, 3)
(128, 128, 3)
(128, 128, 3)
(128, 128, 3)
(128, 128, 3)
(128, 128, 3)
(128, 128, 3)
(128, 128, 3)
(128, 128, 3)
